### Python implementation of the algorithms mentioned in the book "Numerical Optimisation" by Nocedal and Wright, Springer

In [1]:
import numpy as np
import random

#### Algorithm 3.1: Backtracking Line Search

##### Used to analyse the Rosenbrock function, $f(x)$ = $100(x_2-x_1^2)^2+(1-x_1)^2$

In [9]:
x=np.array([-1.2, 1])
alpha=1
c=0.0001
rho=0.1

def f(x): return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

while True:
    B=np.array(([-400*(x[1]-3*x[0]**2), -400*x[0]], [-400*x[0], 200]))
    
    #B=np.eye(2)
    
    delta_f=np.array([-400*(x[0]*x[1]-x[0]**3)-2*(1-x[0]), 200*(x[1]-x[0]**2)])
    print("B=",B)
    print("delta_f=",delta_f)
    p=-(np.linalg.inv(B).dot(delta_f))
    print("p=",p)
    
    x1=x+alpha*p
    L=c*alpha*((delta_f.T).dot(p))
    print("x1=", x1)
    print("L=", L)
    
    alpha=rho*alpha
    print("alpha=", alpha)
    
    if f(x1)>f(x)+L: break
    
    x=x1
alpha

B= [[1328.  480.]
 [ 480.  200.]]
delta_f= [-215.6  -88. ]
p= [0.025 0.38 ]
x1= [-1.175  1.38 ]
L= -0.0038829999999999963
alpha= 0.1
B= [[1104.75  470.  ]
 [ 470.    200.  ]]
delta_f= [-4.64375 -0.125  ]
p= [ 17.4      -40.889375]
x1= [ 0.565     -2.7089375]
L= -0.0007569007812492967
alpha= 0.010000000000000002


0.010000000000000002

#### Algorithm 3.2: Line Search ALgorithm

In [68]:
def LSA(alphaMax, x):
    alpha0=1
    i=1
    alpha1=random.random()
    c1=10**(-4)
    c2=0.9

    f0=f(x)
    g0=g(x)
    H0=H(x)
    B=np.copy(H0)

    alphaold=alpha0
    alphanew=alpha1
    fp=np.copy(f0)
    gp=np.copy(g0)
    
    while True:
        P=-(np.linalg.inv(B)).dot(gp)
        print(i)
        x1=x+alphanew*P
        xnew=np.copy(x1)
        fx=f(x1)
        gx=g(x1)
        Hx=H(x1)

        if (fx>f0+c1*alphanew*((g0.T).dot(P))) or ((i>1) and fx>=fp): return zoom(alphaold, alphanew, x, P)
        if abs((gx.T).dot(P))<=-c2*((g0.T).dot(P)): return alphanew
        if (gx.T).dot(P)>=0: return zoom(alphanew, alphaold, x, P)
        
        alphaold=alphanew
        fp=np.copy(fx)
        gp=np.copy(gx)
        Hp=np.copy(Hx)
        B=np.copy(Hp)
        
        alphanew+=random.uniform(alphanew, alphaMax)
        i+=1

LSA(1.3, np.copy([-3.5, -122.23]))

1
2


0.487412157195802

#### Algorithm 3.3: zoom

In [30]:
def f(x): return 100*(x[1]-x[0]**2)**2+(1-x[0])**2
def g(x): return np.array([-400*(x[0]*x[1]-x[0]**3)-2*(1-x[0]), 200*(x[1]-x[0]**2)])
def H(x): return np.array(([-400*(x[1]-3*x[0]**2)+2, -400*x[0]], [-400*x[0], 200]))

def zoom(alpha1, alpha2, x, P):
    c1=10**(-4)
    c2=0.9
    f0=f(x)
    g0=g(x)
    
    while True:
        alphaj=(alpha1+alpha2)/2
        x1=x+alphaj*P
        fx=f(x1)
        gx=g(x1)
        x2=x+alpha1*P
        fl=f(x2)
        gl=g(x2)
        if fx>f0+c1*alphaj*((g0.T).dot(P)) or fx>=fl: alpha2=alphaj
        else:
            if abs((gx.T).dot(P))<=-c2*((g0.T).dot(P)): return alphaj
            if ((gx.T).dot(P))*(alpha2-alpha1)>=0:alpha2=aplha1
            alpha1=alpha2
x=np.array([1.2, 1.2])
zoom(0.085, 1.0, x, -np.linalg.inv(H(x)).dot(g(x)))

0.5425

#### Algorithm 4.2: Cauchy Point Algorithm

In [10]:
def f(x): return 100*(x[1]-x[0]**2)**2+(1-x[0])**2
def g(x): return np.array([-400*(x[0]*x[1]-x[0]**3)-2*(1-x[0]), 200*(x[1]-x[0]**2)])
def H(x): return np.array(([-400*(x[1]-3*x[0]**2)+2, -400*x[0]], [-400*x[0], 200]))
def m(x, P): 
    B=H(x)
    p1=B.dot(P)
    return f(x)+(g(x).T).dot(P)+0.5*(P.T).dot(p1)

def CPA(x, Delta_max):

    delta=random.uniform(0, Delta_max)
    eta=random.uniform(0, 1/4)
    k=0
    while k<=5000:
        print("k=", k)
        print("x=", x)
        print("delta=", delta)
        B=H(x)
        G=g(x)
        p1=B.dot(G)
        pd=(G.T).dot(p1)
        t=1 if pd<=0 else min(np.linalg.norm(G)**3/(delta*pd), 1)
        P=-(t*delta/np.linalg.norm(G))*G
        print("P=", P)
        
        rho=(f(x)-f(x+P))/(m(x, np.zeros(2))-m(x, P))
        print("rho=", rho)
        print(P)
        if rho<1/4:
            delta=np.linalg.norm(P)/4
        else:
            if rho>3/4 and np.linalg.norm(P)==delta: delta=min(2*delta, Delta_max)
            else: delta=delta
        
        if rho>eta: x+=P
        else:x=x
        k+=1
        
        
CPA(np.array([-30.087, 0.563]), 2)

k= 0
x= [-30.087   0.563]
delta= 0.010345239500537895
P= [0.01034381 0.0001719 ]
rho= 1.0000001183199958
[0.01034381 0.0001719 ]
k= 1
x= [-30.07665619   0.5631719 ]
delta= 0.02069047900107579
P= [0.02068762 0.00034391]
rho= 1.0000004738095007
[0.02068762 0.00034391]
k= 2
x= [-30.05596857   0.56351581]
delta= 0.02069047900107579
P= [0.02068762 0.00034415]
rho= 1.00000047446276
[0.02068762 0.00034415]
k= 3
x= [-30.03528095   0.56385996]
delta= 0.02069047900107579
P= [0.02068761 0.00034439]
rho= 1.0000004751171718
[0.02068761 0.00034439]
k= 4
x= [-30.01459334   0.56420435]
delta= 0.04138095800215158
P= [0.04137522 0.00068925]
rho= 1.0000019047368514
[0.04137522 0.00068925]
k= 5
x= [-29.97321812   0.56489359]
delta= 0.08276191600430316
P= [0.0827504 0.0013804]
rho= 1.0000076532789555
[0.0827504 0.0013804]
k= 6
x= [-29.89046772   0.56627399]
delta= 0.16552383200860632
P= [0.16550068 0.00276844]
rho= 1.0000308908216573
[0.16550068 0.00276844]
k= 7
x= [-29.72496704   0.56904243]
delta= 0.1655

P= [ 0.00585676 -0.00460722]
rho= 1.0080305723879677
[ 0.00585676 -0.00460722]
k= 71
x= [-1.1160221   1.26004163]
delta= 0.5
P= [-0.00242555 -0.00312425]
rho= 0.9986192830495705
[-0.00242555 -0.00312425]
k= 72
x= [-1.11844765  1.25691738]
delta= 0.5
P= [ 0.00576715 -0.00444164]
rho= 1.0078557189369453
[ 0.00576715 -0.00444164]
k= 73
x= [-1.1126805   1.25247574]
delta= 0.5
P= [-0.00239033 -0.00314502]
rho= 0.9986474579973972
[-0.00239033 -0.00314502]
k= 74
x= [-1.11507083  1.24933073]
delta= 0.5
P= [ 0.00568425 -0.00428697]
rho= 1.0076943224404544
[ 0.00568425 -0.00428697]
k= 75
x= [-1.10938658  1.24504375]
delta= 0.5
P= [-0.00235688 -0.00316696]
rho= 0.9986744484025677
[-0.00235688 -0.00316696]
k= 76
x= [-1.11174347  1.24187679]
delta= 0.5
P= [ 0.00560739 -0.00414206]
rho= 1.0075449388265398
[ 0.00560739 -0.00414206]
k= 77
x= [-1.10613608  1.23773473]
delta= 0.5
P= [-0.00232504 -0.00319006]
rho= 0.9987003804952026
[-0.00232504 -0.00319006]
k= 78
x= [-1.10846111  1.23454467]
delta= 0.5


rho= 0.9996685041491211
[-0.0011942  -0.00631419]
k= 172
x= [-0.96113536  0.92649705]
delta= 0.5
P= [ 0.00495166 -0.0009345 ]
rho= 1.0056959304505668
[ 0.00495166 -0.0009345 ]
k= 173
x= [-0.9561837   0.92556255]
delta= 0.5
P= [-0.00115069 -0.00648506]
rho= 0.9996950183257607
[-0.00115069 -0.00648506]
k= 174
x= [-0.95733439  0.91907749]
delta= 0.5
P= [ 0.00498529 -0.00088276]
rho= 1.005720770576688
[ 0.00498529 -0.00088276]
k= 175
x= [-0.9523491   0.91819473]
delta= 0.5
P= [-0.00110388 -0.00666777]
rho= 0.9997220469717544
[-0.00110388 -0.00666777]
k= 176
x= [-0.95345298  0.91152696]
delta= 0.5
P= [ 0.00502211 -0.0008298 ]
rho= 1.0057490215414002
[ 0.00502211 -0.0008298 ]
k= 177
x= [-0.94843087  0.91069715]
delta= 0.5
P= [-0.00105331 -0.00686367]
rho= 0.999749545115259
[-0.00105331 -0.00686367]
k= 178
x= [-0.94948418  0.90383348]
delta= 0.5
P= [ 0.00506241 -0.00077544]
rho= 1.0057809528042785
[ 0.00506241 -0.00077544]
k= 179
x= [-0.94442177  0.90305804]
delta= 0.5
P= [-0.00099844 -0.0070

[0.01256377 0.02139105]
k= 248
x= [0.04105349 0.00712458]
delta= 0.5
P= [ 0.03341313 -0.01810874]
rho= 0.8898038084206402
[ 0.03341313 -0.01810874]
k= 249
x= [ 0.07446662 -0.01098416]
delta= 0.5
P= [0.00893667 0.02174388]
rho= 1.0077456333792127
[0.00893667 0.02174388]
k= 250
x= [0.0834033  0.01075972]
delta= 0.5
P= [ 0.03633861 -0.01410325]
rho= 0.8657841440270074
[ 0.03633861 -0.01410325]
k= 251
x= [ 0.11974191 -0.00334352]
delta= 0.5
P= [0.005518   0.02135835]
rho= 1.0030265068136464
[0.005518   0.02135835]
k= 252
x= [0.12525991 0.01801482]
delta= 0.5
P= [ 0.03885184 -0.00968101]
rho= 0.84184232861111
[ 0.03885184 -0.00968101]
k= 253
x= [0.16411175 0.00833381]
delta= 0.5
P= [0.00247902 0.02045289]
rho= 1.000625276095974
[0.00247902 0.02045289]
k= 254
x= [0.16659077 0.0287867 ]
delta= 0.5
P= [ 0.04141089 -0.00493481]
rho= 0.8156714642884599
[ 0.04141089 -0.00493481]
k= 255
x= [0.20800166 0.02385189]
delta= 0.5
P= [-0.00015478  0.019285  ]
rho= 1.0000024763015027
[-0.00015478  0.01928

rho= 1.0002018474595322
[0.00037711 0.00203115]
k= 374
x= [0.82680307 0.68406692]
delta= 0.12715596181048072
P= [ 0.00076207 -0.0001414 ]
rho= 0.9991733260831471
[ 0.00076207 -0.0001414 ]
k= 375
x= [0.82756514 0.68392552]
delta= 0.12715596181048072
P= [0.00036163 0.00198555]
rho= 1.0001884116003108
[0.00036163 0.00198555]
k= 376
x= [0.82792677 0.68591107]
delta= 0.12715596181048072
P= [ 0.00075134 -0.00013676]
rho= 0.9991844078066004
[ 0.00075134 -0.00013676]
k= 377
x= [0.82867811 0.68577431]
delta= 0.12715596181048072
P= [0.00034702 0.00194212]
rho= 1.0001760494420961
[0.00034702 0.00194212]
k= 378
x= [0.82902514 0.68771643]
delta= 0.12715596181048072
P= [ 0.00074102 -0.00013233]
rho= 0.9991950761990832
[ 0.00074102 -0.00013233]
k= 379
x= [0.82976616 0.6875841 ]
delta= 0.12715596181048072
P= [0.00033322 0.00190071]
rho= 1.0001646550087768
[0.00033322 0.00190071]
k= 380
x= [0.83009939 0.68948481]
delta= 0.12715596181048072
P= [ 0.00073109 -0.0001281 ]
rho= 0.9992053575487051
[ 0.000731

...

rho= 1.0000018411737321
[-4.61057734e-06  1.39132988e-05]
k= 3788
x= [0.99427533 0.98857199]
delta= 0.12715596181048072
P= [1.38946210e-05 4.60443052e-06]
rho= 0.9999832917966971
[1.38946210e-05 4.60443052e-06]
k= 3789
x= [0.99428922 0.9885766 ]
delta= 0.12715596181048072
P= [-4.60326731e-06  1.38899435e-05]
rho= 1.0000018383719718
[-4.60326731e-06  1.38899435e-05]
k= 3790
x= [0.99428462 0.98859049]
delta= 0.12715596181048072
P= [1.38719660e-05 4.59735187e-06]
rho= 0.9999833189265925
[1.38719660e-05 4.59735187e-06]
k= 3791
x= [0.99429849 0.98859509]
delta= 0.12715596181048072
P= [-4.59596879e-06  1.38666293e-05]
rho= 1.0000018355417877
[-4.59596879e-06  1.38666293e-05]
k= 3792
x= [0.9942939  0.98860895]
delta= 0.12715596181048072
P= [1.38493488e-05 4.59028366e-06]
rho= 0.9999833459956378
[1.38493488e-05 4.59028366e-06]
k= 3793
x= [0.99430775 0.98861354]
delta= 0.12715596181048072
P= [-4.58868176e-06  1.38433563e-05]
rho= 1.0000018327277425
[-4.58868176e-06  1.38433563e-05]
k= 3794
x= [

k= 3841
x= [0.9945253  0.98904753]
delta= 0.12715596181048072
P= [-4.41719403e-06  1.32969331e-05]
rho= 1.000001766397627
[-4.41719403e-06  1.32969331e-05]
k= 3842
x= [0.99452088 0.98906082]
delta= 0.12715596181048072
P= [1.32960182e-05 4.41692929e-06]
rho= 0.9999840084927832
[1.32960182e-05 4.41692929e-06]
k= 3843
x= [0.99453418 0.98906524]
delta= 0.12715596181048072
P= [-4.41018853e-06  1.32746625e-05]
rho= 1.0000017636856318
[-4.41018853e-06  1.32746625e-05]
k= 3844
x= [0.99452977 0.98907852]
delta= 0.12715596181048072
P= [1.32743614e-05 4.41012755e-06]
rho= 0.9999840344330672
[1.32743614e-05 4.41012755e-06]
k= 3845
x= [0.99454304 0.98908293]
delta= 0.12715596181048072
P= [-4.40319406e-06  1.32524310e-05]
rho= 1.0000017609882519
[-4.40319406e-06  1.32524310e-05]
k= 3846
x= [0.99453864 0.98909618]
delta= 0.12715596181048072
P= [1.32527407e-05 4.40333588e-06]
rho= 0.9999840603249498
[1.32527407e-05 4.40333588e-06]
k= 3847
x= [0.99455189 0.98910058]
delta= 0.12715596181048072
P= [-4.39

P= [-4.19840771e-06  1.26033063e-05]
rho= 1.000001681528587
[-4.19840771e-06  1.26033063e-05]
k= 3906
x= [0.99479808 0.9896128 ]
delta= 0.12715596181048072
P= [1.26205855e-05 4.20419924e-06]
rho= 0.9999848174863026
[1.26205855e-05 4.20419924e-06]
k= 3907
x= [0.9948107 0.989617 ]
delta= 0.12715596181048072
P= [-4.19174699e-06  1.25822508e-05]
rho= 1.000001678952782
[-4.19174699e-06  1.25822508e-05]
k= 3908
x= [0.99480651 0.98962958]
delta= 0.12715596181048072
P= [1.26000525e-05 4.19771294e-06]
rho= 0.9999848420921911
[1.26000525e-05 4.19771294e-06]
k= 3909
x= [0.99481911 0.98963378]
delta= 0.12715596181048072
P= [-4.18509677e-06  1.25612321e-05]
rho= 1.0000016763648127
[-4.18509677e-06  1.25612321e-05]
k= 3910
x= [0.99481492 0.98964634]
delta= 0.12715596181048072
P= [1.25795536e-05 4.19123628e-06]
rho= 0.9999848666641412
[1.25795536e-05 4.19123628e-06]
k= 3911
x= [0.9948275  0.98965053]
delta= 0.12715596181048072
P= [-4.17845704e-06  1.25402502e-05]
rho= 1.0000016737780226
[-4.17845704e

delta= 0.12715596181048072
P= [-3.96512238e-06  1.18679988e-05]
rho= 1.0000015907253068
[-3.96512238e-06  1.18679988e-05]
k= 3978
x= [0.99509294 0.99020011]
delta= 0.12715596181048072
P= [1.19024747e-05 3.97667256e-06]
rho= 0.9999856779985511
[1.19024747e-05 3.97667256e-06]
k= 3979
x= [0.99510484 0.99020408]
delta= 0.12715596181048072
P= [-3.95882959e-06  1.18482249e-05]
rho= 1.0000015882680378
[-3.95882959e-06  1.18482249e-05]
k= 3980
x= [0.99510088 0.99021593]
delta= 0.12715596181048072
P= [1.18831334e-05 3.97052512e-06]
rho= 0.9999857011716141
[1.18831334e-05 3.97052512e-06]
k= 3981
x= [0.99511276 0.9902199 ]
delta= 0.12715596181048072
P= [-3.95254673e-06  1.18284855e-05]
rho= 1.0000015858117042
[-3.95254673e-06  1.18284855e-05]
k= 3982
x= [0.99510881 0.99023173]
delta= 0.12715596181048072
P= [1.18638242e-05 3.96438687e-06]
rho= 0.9999857243384502
[1.18638242e-05 3.96438687e-06]
k= 3983
x= [0.99512067 0.99023569]
delta= 0.12715596181048072
P= [-3.94627379e-06  1.18087803e-05]
rho= 1

KeyboardInterrupt: 

#### Algorithm 4.3: CG-Steihaug Algorithm

##### The trust region subproblem is as follows:
\begin{equation} 
min\space \space m(P)=f+g^TP+\frac{1}{2}P^TBP
\end{equation}
##### subject to:
\begin{equation} ||P||=\Delta \end{equation}

##### So,
\begin{equation}
\sqrt{(P_j^0+\tau d_j^0)^2 + (P_j^1+\tau d_j^1)^2+ ... +(P_j^n+\tau d_j^n)^2}=\Delta \\
\Rightarrow (P_j^0+\tau d_j^0)^2 + (P_j^1+\tau d_j^1)^2+ ... +(P_j^n+\tau d_j^n)^2=\Delta^2\\
\Rightarrow (P_j^0)^2+2P_j^0\tau d_j^0+(\tau d_j^0)^2+(P_j^1)^2+2P_j^1\tau d_j^1+(\tau d_j^1)^2+...+(P_j^n)^2+2P_j^n\tau d_j^n+(\tau d_j^n)^2=\Delta^2\\
\Rightarrow \tau^2 ((d_j^0)^2+(d_j^1)^2+...+(d_j^n)^2)+2\tau (P_j^0d_j^0+P_j^1d_j^1+...+P_j^nd_j^n)+(P_j^0)^2+(P_j^1)^2+...+(P_j^n)^2-\Delta^2=0\\
\Rightarrow \tau^2 ||d_j||^2+2\tau P_j^Td_j+||P_j||^2-\Delta^2=0\\
\Rightarrow \tau=\frac{-P_j^Td_j \frac{+}{} \sqrt{(P_j^Td_j)^2-||d_j||^2(||P_j||^2-\Delta^2)}}{||d_j||^2}
\end{equation}

In [3]:
def f(x): return 100*(x[1]-x[0]**2)**2+(1-x[0])**2
def g(x): return np.array([-400*(x[0]*x[1]-x[0]**3)-2*(1-x[0]), 200*(x[1]-x[0]**2)])
def H(x): return np.array(([-400*(x[1]-3*x[0]**2)+2, -400*x[0]], [-400*x[0], 200]))

def CGS(Delta, x):
    epsilon=10**(-6)
    r=g(x)
    P=np.zeros(len(r))
    r0=np.copy(r)
    d=-np.copy(r)
    B=H(x)
    j=0
    if np.linalg.norm(r0)<epsilon: return P
    
    while True:
        p1=B.dot(d)
        alpha=(d.T).dot(p1)
        if alpha<=0:
            A=-(P.T).dot(d)
            C=((P.T).dot(d))**2
            D=np.linalg.norm(d)**2
            E=np.linalg.norm(P)**2-Delta**2
            alpha=(A+np.sqrt(C-D*E))/D

            return P + alpha*d
        else:
            alpha=((r.T).dot(r))/alpha
            P1=np.copy(P)
            P+=alpha*d
        
            if np.linalg.norm(P)>=Delta:
                A=-(P1.T).dot(d)
                C=((P1.T).dot(d))**2
                D=np.linalg.norm(d)**2
                E=np.linalg.norm(P1)**2-Delta**2
                alpha=(A+np.sqrt(C-D*E))/D

                return P1 + alpha*d
        
        r1=np.copy(r)
        r+=alpha*(B.dot(d))
        
        if np.linalg.norm(r)<epsilon*np.linalg.norm(r0): return P
        
        beta=((r.T).dot(r))/((r1.T).dot(r1))
        d1=np.copy(d)
        d=r+beta*d1
    
        print("P=",P)
        
        j+=1
        print(j)
        
CGS(1.5, np.array([1.2, -0.5]))
    

P= [-0.46233356  0.19255627]
1
P= [-0.77149824 -0.33422452]
2
P= [-0.91929185 -0.49909225]
3
P= [-1.0240319  -0.59281691]
4
P= [-1.10814938 -0.65741695]
5
P= [-1.17985734 -0.70631727]
6
P= [-1.24313913 -0.74543611]
7


array([-1.28705198, -0.7703877 ])

#### Dogleg implementation

In [15]:
def f(x): return 100*(x[1]-x[0]**2)**2+(1-x[0])**2
def g(x): return np.array([-400*(x[0]*x[1]-x[0]**3)-2*(1-x[0]), 200*(x[1]-x[0]**2)])
def H(x): return np.array(([-400*(x[1]-3*x[0]**2)+2, -400*x[0]], [-400*x[0], 200]))
def m(x, P): 
    B=H(x)
    p1=B.dot(P)
    return f(x)+(g(x).T).dot(P)+0.5*(P.T).dot(p1)

def dogleg(Delta, Pg, Pn):
    if Delta<=np.linalg.norm(Pg): return (Delta/np.linalg.norm(Pg))*Pg
    elif Delta>=np.linalg.norm(Pn): return Pn
    else:
        A=np.linalg.norm(Pg)**2
        F=np.linalg.norm(Pn)**2
        C=np.linalg.norm(Pg-Pn)**2
        D=(A+F-C)/2
        E=(F-Delta**2)/(F-D+np.sqrt(D**2-A*F+Delta**2*C))
        return E*Pg+(1-E)*Pn

def DI(x, Delta_max):

    delta=random.uniform(0, Delta_max)
    eta=random.uniform(0, 1/4)
    k=0
    while k<=5000:
        print("k=", k)
        print("x=", x)
        print("delta=", delta)
        B=H(x)
        G=g(x)
        p1=B.dot(G)
        Pg=-G*((G.T).dot(G))/((G.T).dot(p1))
        Pn=-np.linalg.inv(B).dot(G)
        P=dogleg(delta, Pg, Pn)
        print("P=", P)
        
        rho=(f(x)-f(x+P))/(m(x, np.zeros(2))-m(x, P))
        print("rho=", rho)
        print(P)
        if rho<1/4:
            delta=np.linalg.norm(P)/4
        else:
            if rho>3/4 and np.linalg.norm(P)==delta: delta=min(2*delta, Delta_max)
            else: delta=delta
        
        if rho>eta: x+=P
        else:x=x
        k+=1
        
        
DI(np.array([1.2, 1.2]), 2)

k= 0
x= [1.2 1.2]
delta= 1.40143198446331
P= [-0.00408163  0.23020408]
rho= 1.000138806551976
[-0.00408163  0.23020408]
k= 1
x= [1.19591837 1.43020408]
delta= 1.40143198446331
P= [-0.19526775 -0.46703191]
rho= -2.7969632791942876
[-0.19526775 -0.46703191]
k= 2
x= [1.19591837 1.43020408]
delta= 0.12655243375657896
P= [-0.04903883 -0.11666495]
rho= 0.9835194137884729
[-0.04903883 -0.11666495]
k= 3
x= [1.14687953 1.31353913]
delta= 0.12655243375657896
P= [-0.05133913 -0.11567114]
rho= 1.034387080394262
[-0.05133913 -0.11567114]
k= 4
x= [1.0955404  1.19786799]
delta= 0.12655243375657896
P= [-0.0534518  -0.11471017]
rho= 1.0851484810115826
[-0.0534518  -0.11471017]
k= 5
x= [1.04208861 1.08315782]
delta= 0.2531048675131579
P= [-0.02701158 -0.05350608]
rho= 1.184792841287131
[-0.02701158 -0.05350608]
k= 6
x= [1.01507702 1.02965174]
delta= 0.2531048675131579
P= [-0.01315707 -0.02598126]
rho= 1.0884888955471943
[-0.01315707 -0.02598126]
k= 7
x= [1.00191995 1.00367047]
delta= 0.2531048675131579


F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


rho= nan
[-0. -0.]
k= 92
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 93
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 94
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 95
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 96
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 97
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 98
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 99
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 100
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 101
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 102
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 103
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 104
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k

k= 228
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 229
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 230
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 231
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 232
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 233
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 234
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 235
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 236
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 237
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 238
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 239
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 240
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 241
x= [1

x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 365
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 366
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 367
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 368
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 369
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 370
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 371
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 372
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 373
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 374
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 375
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 376
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 377
x= [1. 1.]
d

...

P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3181
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3182
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3183
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3184
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3185
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3186
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3187
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3188
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3189
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3190
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3191
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3192
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3193
x= [1. 1.]
delta= 0.2531048675131579

delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3363
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3364
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3365
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3366
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3367
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3368
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3369
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3370
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3371
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3372
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3373
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3374
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3375
x= [1. 1.]

k= 3544
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3545
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3546
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3547
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3548
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3549
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3550
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3551
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3552
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3553
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3554
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3555
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3556
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]

rho= nan
[-0. -0.]
k= 3726
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3727
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3728
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3729
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3730
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3731
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3732
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3733
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3734
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3735
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3736
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3737
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]
rho= nan
[-0. -0.]
k= 3738
x= [1. 1.]
delta= 0.2531048675131579
P= [-0. -0.]

KeyboardInterrupt: 

#### Algorithm 4.4: Exact Trust Region Algorithm

In [19]:
def ETR(Delta, B, G):
    Lambda=0
    epsilon=10**(-6)
    P=np.linalg.inv(B).dot(G)
    while abs(np.linalg.norm(P)-Delta)>epsilon and Lambda>=0:
        P1=-np.linalg.inv(B+Lambda*np.eye(len(P))).dot(P)
        Phi=np.linalg.norm(P)-Delta
        Phi1=((P.T).dot(P1))/np.linalg.norm(P)
        Lambda-=(Phi*np.linalg.norm(P))/(Phi1*Delta)
        P=-np.linalg.inv(B+Lambda*np.eye(len(G))).dot(G)
    if Lambda<0: P=-np.linalg.inv(B).dot(G)